In [8]:
import ipyparallel as ipp
rc = ipp.Client()
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [9]:
dview = rc[:] # use all engines
lview = rc.load_balanced_view() # default load-balanced view

In [10]:
def get_K(d): 
    import numpy as np
    import numpy.matlib
    Sniter = np.array(range(d))
    Sniter = np.matlib.repmat(Sniter,d,1)
    Sniter = Sniter.reshape((1,d**2))
    Smiter = np.array(range(d))
    Smiter = np.matlib.repmat(Smiter,d,1)
    Smiter = Smiter.T.reshape((1,d**2))

    # starting with all operators except (0,0)
    Sniter_all = Sniter[0][1:]
    Smiter_all = Smiter[0][1:]

    Sniter = [1] + [ii for ii in range(d)]
    Smiter = [0] + [1 for ii in range(d)]

    op_All = [(Smiter_all[ii], Sniter_all[ii]) for ii in range(len(Sniter_all))]
    op_Current = [(Sniter[ii], Smiter[ii]) for ii in range(d)]
    
    n_eigVals = [d/np.gcd(W[0], np.gcd(W[1], d)) for W in op_All]
    #for ii, ee in zip(op_All, n_eigVals):
    #    print(ii, ee)
        
    op_tempKept = [op_All[ii] for ii in range(len(op_All)) if n_eigVals[ii] == d] # all full-rank operators
    
    op_finalKept = [op_tempKept[0]]
    for W in op_tempKept:
        L = [(K[0]*W[1] - K[1]*W[0])%d for K in op_finalKept]
        if 0 not in L:
            op_finalKept.append(W)
        #print(op_finalKept)
    
    #print(len(op_finalKept))
    Sniter = [W[0] for W in op_finalKept]
    Smiter = [W[1] for W in op_finalKept]
    #print(Sniter)



    eigRel = np.zeros((len(Sniter),d**2))
    for kk in range(len(Sniter)):
        appOp = 0;
        for nn in range(d):
            for mm in range(d):
                eigRel[kk,appOp] = (Smiter[kk]*nn - Sniter[kk]*mm)%d#(mm*g - nn*t)%d
                appOp +=1
    eig_Rel = {(Sniter[ii], Smiter[ii]): list(eigRel[ii]) for ii in range(len(Sniter))}


    idx_opers = list(eig_Rel.keys()) # indices of Weyl operators generators of input states
    out_rels = list(eig_Rel.values()) # corresponding output relations
    rel_matrix = np.matrix([np.array(row)==kk for row in out_rels for kk in range(d)], dtype = int)

    #print(np.sum(rel_matrix,axis = 1))

    #print("Total Inputs: ", len(idx_opers))
    #print("Rank of A: ", np.linalg.matrix_rank(rel_matrix),"\nShould be: ", d**2)
    print([d, len(idx_opers), len(rel_matrix)/d, d**2-np.linalg.matrix_rank(rel_matrix)])
    return([d, len(idx_opers), len(rel_matrix)/d, d**2-np.linalg.matrix_rank(rel_matrix)])

In [11]:
asdf = lview.map_async(get_K, [ii for ii in range(2, 101)], chunksize = 2)

# initialize a stdout0 array for comparison
stdout0 = asdf.stdout

while not asdf.ready():
    # check if stdout changed for any kernel
    if asdf.stdout != stdout0:
        for i in range(0,len(asdf.stdout)):
            if asdf.stdout[i] != stdout0[i]:
                # print only new stdout's without previous message and remove '\n' at the end
                print('kernel ' + str(i) + ': ' + asdf.stdout[i][len(stdout0[i]):-1])
                stdout0 =  asdf.stdout
    else:
        continue


asdf.get()

kernel 4: [10, 18, 18.0, 0]
[11, 12, 12.0, 0]
kernel 0: [2, 3, 3.0, 0]
[3, 4, 4.0, 0]
kernel 6: [14, 24, 24.0, 0]
[15, 24, 24.0, 0]
kernel 7: [16, 24, 24.0, 0]
kernel 7: [17, 18, 18.0, 0]
kernel 8: [18, 36, 36.0, 0]
kernel 9: [20, 36, 36.0, 0]
kernel 8: [19, 20, 20.0, 0]
kernel 10: [22, 36, 36.0, 0]
kernel 9: [21, 32, 32.0, 0]
kernel 11: [24, 48, 48.0, 0]
kernel 10: [23, 24, 24.0, 0]
kernel 13: [28, 48, 48.0, 0]
kernel 11: [25, 30, 30.0, 0]
kernel 12: [27, 36, 36.0, 0]
kernel 14: [30, 72, 72.0, 0]
kernel 15: [32, 48, 48.0, 0]
kernel 16: [34, 54, 54.0, 0]
kernel 13: [29, 30, 30.0, 0]
kernel 14: [31, 32, 32.0, 0]
kernel 18: [38, 60, 60.0, 0]
kernel 15: [33, 48, 48.0, 0]
kernel 17: [36, 72, 72.0, 0]
kernel 16: [35, 48, 48.0, 0]
kernel 19: [40, 72, 72.0, 0]
kernel 21: [44, 72, 72.0, 0]
kernel 18: [39, 56, 56.0, 0]
kernel 17: [37, 38, 38.0, 0]
kernel 22: [46, 72, 72.0, 0]
kernel 20: [42, 96, 96.0, 0]
kernel 19: [41, 42, 42.0, 0]
kernel 23: [48, 96, 96.0, 0]
kernel 20: [43, 44, 44.0, 0]
kern

[[2, 3, 3.0, 0],
 [3, 4, 4.0, 0],
 [4, 6, 6.0, 0],
 [5, 6, 6.0, 0],
 [6, 12, 12.0, 0],
 [7, 8, 8.0, 0],
 [8, 12, 12.0, 0],
 [9, 12, 12.0, 0],
 [10, 18, 18.0, 0],
 [11, 12, 12.0, 0],
 [12, 24, 24.0, 0],
 [13, 14, 14.0, 0],
 [14, 24, 24.0, 0],
 [15, 24, 24.0, 0],
 [16, 24, 24.0, 0],
 [17, 18, 18.0, 0],
 [18, 36, 36.0, 0],
 [19, 20, 20.0, 0],
 [20, 36, 36.0, 0],
 [21, 32, 32.0, 0],
 [22, 36, 36.0, 0],
 [23, 24, 24.0, 0],
 [24, 48, 48.0, 0],
 [25, 30, 30.0, 0],
 [26, 42, 42.0, 0],
 [27, 36, 36.0, 0],
 [28, 48, 48.0, 0],
 [29, 30, 30.0, 0],
 [30, 72, 72.0, 0],
 [31, 32, 32.0, 0],
 [32, 48, 48.0, 0],
 [33, 48, 48.0, 0],
 [34, 54, 54.0, 0],
 [35, 48, 48.0, 0],
 [36, 72, 72.0, 0],
 [37, 38, 38.0, 0],
 [38, 60, 60.0, 0],
 [39, 56, 56.0, 0],
 [40, 72, 72.0, 0],
 [41, 42, 42.0, 0],
 [42, 96, 96.0, 0],
 [43, 44, 44.0, 0],
 [44, 72, 72.0, 0],
 [45, 72, 72.0, 0],
 [46, 72, 72.0, 0],
 [47, 48, 48.0, 0],
 [48, 96, 96.0, 0],
 [49, 56, 56.0, 0],
 [50, 90, 90.0, 0],
 [51, 72, 72.0, 0],
 [52, 84, 84.0, 0]